In [ ]:
import numpy as np
import sys
import cv2
import pandas as pd 
from matplotlib import pyplot as plt
(CV2_VERSION, minor_ver, subminor_ver) = (cv2.__version__).split('.')
print('openCV version - ', CV2_VERSION)

In [ ]:
CSV_NAME = 'list.csv'
# out images will have dimesions of (width, height)
w = 300 
h = 300
DIM = (w, h)

###### Cap Functions

In [ ]:
def open_video(video):
    return cv2.VideoCapture(video)

def check_video(cap):
    success, frame = cap.read()
    if success == False:
        print("Video doesn't exit")
        exit(1)

def close_cap(cap):
    cap.release()
    #cv2.destroyAllWindows() #Close viewing of frames

##### Generic functions

In [ ]:
def getFrameRate(video):
    fps = 30 # default frame rate - this values should be replace in the next if/else evaluation
    if int(CV2_VERSION)  < 3 :
        fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
    else :
        fps = video.get(cv2.CAP_PROP_FPS)
    return fps

# start is in mili seconds
# curr is in seconds
# end is in mili seconds or is the word 'END'
def between(start,curr,end):
    
    if end == 'END':
        nend = 86400000.0 # BIGINT Number, represents 1 day... in python 3 there is no INT Max, in pyhton 2 it is 9223372036854775807, 
    else:
        nend = float(end)
        
    start_ms = start / 1000
    end_ms = nend / 1000
    
    return start_ms <= curr <= end_ms

def generate_out_videoname(vid):
    
    # check if the Video name is already formated
    if "video-" in vid:
        return vid.split('.')[0]
    
    out_video = 'video-00001' # default video name
    
    try:
        collision_number = vid.split('.')[0]
        numb = collision_number.split('collision')[1]
        s_numb = str(numb)
        while len(s_numb) < 5:
            s_numb = '0' + s_numb
        return f"video-{s_numb}"
    except:
        print(f"Exception generating new video name; returning name {out_video}")
        return out_video
    
def generate_framename(video_num,pos_frame):
    
    s_outvid = str(video_num)
    s_frame = str(pos_frame)
    
    while len(s_outvid) < 5:
        s_outvid = '0' + s_outvid
        
    while len(s_frame) < 5:
        s_frame = '0' + s_frame
    
    return f"video-{s_outvid}-frame-{s_frame}"

def generate_video_num(out_videoname):
    return int(out_videoname.split('-')[1])



##### CSV functions
Inside the list.csv each row represent a video that you will want to write a vector
Example of a line inside of list.csv: "video.avi start_time(ms) end_time(ms) vector"
Multiple times can be added in the form of "video.avi start_time1(ms) end_time1(ms) vector1 start_time2(ms) end_time2(ms) vector2"

In [ ]:
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
# header=None means no header is to be considered
# Format => video.avi start_time1(ms) end_time1(ms) vector1 start_time2(ms) end_time2(ms) vector2 (...n)
video_vector_list = pd.read_csv(CSV_NAME,header=None) 
# Preview the first 5 lines of the loaded data
video_vector_list.head()

In [ ]:
VIDEO_NAME = ''
start_times = []
end_times = []
escape_vectors = []
for index, row in video_vector_list.iterrows():
    VIDEO_NAME = row[0]
    
    # iterate the START END VECTOR
    n_blocks = int((len(row) - 1 ) / 3)  # length of rows, minus the first (video_name), divided by 3 (START, END, VECTOR)    
    for i in range(n_blocks):
        offset = (i + 1) * 3 - 2
        start_times.append(row[offset])
        end_times.append(row[offset + 1])
        escape_vectors.append(row[offset + 2])
        
    print(VIDEO_NAME)
    print(start_times)
    print(end_times)
    print(escape_vectors)
        
    

##### Handle video

In [ ]:
video = open_video(VIDEO_NAME)
while not video.isOpened():
    video = cv2.VideoCapture(VIDEO_NAME)
    cv2.waitKey(1000)
    print("Wait for the header")

In [ ]:
pos_frame = video.get(cv2.CAP_PROP_POS_FRAMES) # get first frame counter
fps = getFrameRate(video) # get frame rate
fps = round(fps, 2) # round to 2 decimal places;
stop_loop_by_not_ready = 0
MAX_STOP_LOOP_COUNTER = 5
last_pos_frame = 0 # pos frame counter. used to compare with previous 
out_videoname = generate_out_videoname(VIDEO_NAME)
video_number = generate_video_num(out_videoname)
consecutive_counter = 0
MAX_CONSECUTIVE = 100

df = pd.DataFrame(columns=['file', 'collision', 'xmin', 'xmax', 'ymin', 'ymax', 'zmin', 'zmax'])


print(f'Processing video "{VIDEO_NAME}" that is {fps} fps')

while True:
        
    flag, frame = video.read()    
    if flag:
        # The frame is ready and already captured
        last_pos_frame = pos_frame = video.get(cv2.CAP_PROP_POS_FRAMES) # get frame counter
        
        time_elapsed = round((pos_frame / fps ) ,2)# this calculates the time elapsed in second        
        
        curr_frame_name = generate_framename(video_number, int(pos_frame))
        isCollision = False
        for index, val in enumerate(start_times):
            
            if between(val,time_elapsed,end_times[index]):
                vec = escape_vectors[index].split(' ')
                new_row = {
                    'file' : curr_frame_name,
                    'collision': 1, 
                    'xmin': vec[0],
                    'xmax': vec[1], 
                    'ymin': vec[2],
                    'ymax': vec[3],
                    'zmin': vec[4],
                    'zmax': vec[5]
                }
                isCollision = True
            else:
                new_row = {
                    'file' : curr_frame_name,
                    'collision': 0, 
                    'xmin': 0,
                    'xmax': 0, 
                    'ymin': 0,
                    'ymax': 0, 
                    'zmin': 0, 
                    'zmax': 0
                }
                
        isSaveData = True
            
        if isCollision:
            print(f'Frame {pos_frame} at time {time_elapsed} s -> Collision')
            consecutive_counter += 1
            if consecutive_counter > MAX_CONSECUTIVE:
                print(f'Too much consecutive found ({consecutive_counter}), jump to the next frame and ignore this one.')
                isSaveData = False
        else:
            print(f'Frame {pos_frame} at time {time_elapsed} s -> NO collision')
            consecutive_counter = 0
        
        if isSaveData:             
            df = df.append(new_row, ignore_index=True)
            resized_frame = cv2.resize(frame, DIM, interpolation = cv2.INTER_AREA)
            # save image to file
            cv2.imwrite(f"./images/{curr_frame_name}.png", resized_frame) 
            # plot image
            plt.imshow(resized_frame) # plot/display current frame
            plt.pause(0.05) # give some time to matplot
            plt.show() # force display inside the loop

        
    #### THE CODE BELLOW IS JUST FOR HANDLING VIDEO NONSENSES...
    else:
        # The next frame is not ready, so we try to read it again
        video.set(cv2.CAP_PROP_POS_FRAMES, pos_frame-1)
        # It is better to wait for a while for the next frame to be ready
        cv2.waitKey(1000)
        if last_pos_frame == pos_frame:
            stop_loop_by_not_ready += 1 # increment loop not ready counter
        else:
            stop_loop_by_not_ready = 0 # reset loop not ready counter

        print(f"frame is not ready, counter is at {stop_loop_by_not_ready}")


    if stop_loop_by_not_ready >= MAX_STOP_LOOP_COUNTER:
        # for some reason, sometimes it was not stopping. This if enforces that stop
        print(f"Interruped because last_pos_frame == pos_frame with value pos_frame= {last_pos_frame}")
        break
    if cv2.waitKey(10) == 27:
        break
    if video.get(cv2.CAP_PROP_POS_FRAMES) == video.get(cv2.CAP_PROP_FRAME_COUNT):
        # If the number of captured frames is equal to the total number of frames,
        # we stop
        break
        
        
# dropping duplicate values 
df.drop_duplicates(keep='first',inplace=True)
df.to_excel(f'dataframes/{out_videoname}.xlsx',index=False)  

### Finally print the DataFrame df generated.
#### Note : the file images were saved under ./images 

In [ ]:
df